In [1]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
 
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import keras_metrics as km
 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from tqdm import tqdm
import math   # for mathematical operations
import random
from glob import glob
from keras.utils import to_categorical

Using TensorFlow backend.


In [13]:
img_height , img_width = 244, 244
seq_len = 70
IMAGE_CHANNELS = 3
batch = 8

INPUT_SHAPE = (seq_len, img_height, img_width, IMAGE_CHANNELS)

classes = ["Fight", "NonFight"]

In [3]:
TRAIN_FOLDER_PATH = r'C:\\Users\djcoo\Documents\LYIT\Final\project\RWF-2000_test\train'
TEST_FOLDER_PATH = r'C:\\Users\djcoo\Documents\LYIT\Final\project\RWF-2000_test\val'

def listDir(dir):
    catgeory = {}
    dirNames = os.listdir(dir)
    for dirName in dirNames:
        catgeory[dirName] = []
#         print('Directory Name : '+ dirName)
        dir_path = os.path.abspath(os.path.join(dir, dirName))
#         print('Directory Path : '+ dir_path)
        fileNames = os.listdir(dir_path)
        for fileName in fileNames:
            file_path = os.path.abspath(os.path.join(dir_path, fileName))
#             print('File Path : '+ file_path )
            catgeory[dirName].append(file_path)
    return catgeory

LIST_OF_TRAIN_CATEGORY = listDir(TRAIN_FOLDER_PATH)
LIST_OF_TEST_CATEGORY = listDir(TEST_FOLDER_PATH)

In [4]:
def get_video_frames(path_arr, flag):
    label_arr = ["Fight", "NonFight"]
    for label in label_arr:
        image_count = 0
        for video_path in tqdm(path_arr[label]):
#             video_name = video_path.split('\\').pop().split(".")[0]
        #     print(video_name)
            cap = cv2.VideoCapture(video_path)   # capturing the video from the given path
            frameRate = cap.get(5) #frame rate
            frame_count = 0
            while(cap.isOpened()):
                frameId = cap.get(1) #current frame number
                ret, frame = cap.read()
                if (ret != True):
                    break
                if (frameId % math.floor(frameRate) == 0):
                    # storing the frames in a new folder named train_1
                    TRAIN_FRAME_PATH = "C:\\Users\\djcoo\\Documents\\LYIT\\Final\\project\\frames\\"+ flag +"\\" + label + "_"
#                     filename = r"%s" %TRAIN_FRAME_PATH + video_name +"_frame%d.jpg" %frame_count;
                    filename = r"%s" %TRAIN_FRAME_PATH + "%d"%image_count +"_frame%d.jpg" %frame_count
#                     print(filename)
                    frame_count+=1
                    cv2.imwrite(filename, frame)
            image_count+=1
            cap.release()

get_video_frames(LIST_OF_TRAIN_CATEGORY, "train")
get_video_frames(LIST_OF_TEST_CATEGORY, "test")

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.46it/s]


In [5]:
images = glob(r"C:\\Users\\djcoo\\Documents\\LYIT\\Final\\project\\frames\\train\\*.jpg")
# creating an empty list
random.shuffle(images)

train_image = []
train_class = []

# for loop to read and store frames
for img_path in tqdm(images):
    # loading the image and keeping the target size as (224,224,3)
#     img = image.load_img(img_path, target_size=(224,224,3))
#     # converting it to array
#     img = image.img_to_array(img)
#     # normalizing the pixel value
#     img = img/255
#     # appending the image to the train_image list
#     print(img_path) 
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    
    train_image.append(image)
    train_class.append(img_path.split("\\").pop().split("_")[0])
    
# converting the list to numpy array
X_train = np.array(train_image)

# shape of the array
X_train.shape

100%|███████████████████████████████████████████████████████████████████████████████| 150/150 [00:00<00:00, 442.55it/s]


(150, 224, 224, 3)

In [6]:
images = glob(r"C:\\Users\\djcoo\\Documents\\LYIT\\Final\\project\\frames\\test\\*.jpg")
# creating an empty list
random.shuffle(images)

test_image = []
test_class = []

# for loop to read and store frames
for img_path in tqdm(images):
#     # loading the image and keeping the target size as (224,224,3)
#     img = image.load_img(img_path, target_size=(224,224,3))
#     # converting it to array
#     img = image.img_to_array(img)
#     # normalizing the pixel value
#     img = img/255
    # appending the image to the train_image list
#     print(img_path) 
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image = cv2.resize(image, (224, 224))
    
    test_image.append(image)
    test_class.append(img_path.split("\\").pop().split("_")[0])
    
# converting the list to numpy array
X_test = np.array(test_image)

# shape of the array
X_test.shape

100%|█████████████████████████████████████████████████████████████████████████████████| 95/95 [00:00<00:00, 113.36it/s]


(95, 224, 224, 3)

In [7]:
lb = LabelBinarizer()
y_train = to_categorical(lb.fit_transform(train_class))
y_test = to_categorical(lb.fit_transform(test_class))

# print(len(y_train))
# print(len(y_test))

In [14]:
model = Sequential()
model.add(ConvLSTM2D(filters = 40, kernel_size = (3, 3), return_sequences = False, data_format = "channels_last", batch_input_shape = INPUT_SHAPE))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(6, activation = "softmax"))

In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_2 (ConvLSTM2D)  (8, 242, 242, 40)         62080     
_________________________________________________________________
dropout_3 (Dropout)          (8, 242, 242, 40)         0         
_________________________________________________________________
flatten_2 (Flatten)          (8, 2342560)              0         
_________________________________________________________________
dense_3 (Dense)              (8, 512)                  1199391232
_________________________________________________________________
dropout_4 (Dropout)          (8, 512)                  0         
_________________________________________________________________
dense_4 (Dense)              (8, 6)                    3078      
Total params: 1,199,456,390
Trainable params: 1,199,456,390
Non-trainable params: 0
____________________________________

In [16]:
opt = keras.optimizers.SGD(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [17]:
earlystop = EarlyStopping(patience=7)
callbacks = [earlystop]
 
history = model.fit(x = X_train, y = y_train, epochs=40 , shuffle=True, validation_split=0.2, callbacks=callbacks)

ValueError: Error when checking input: expected conv_lst_m2d_2_input to have 5 dimensions, but got array with shape (150, 224, 224, 3)

In [ ]:
y_pred = np.argmax(y_pred, axis = 1)
y_test = np.argmax(y_test, axis = 1)
 
print(classification_report(y_test, y_pred))